# Imports

In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling
)
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig
import torch
import wandb

/home/super_admin/hasindu/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Configs

In [52]:
data_path = "../data/qna/"
train_data_path = data_path + "train.csv"
val_data_path = data_path + "val.csv"

max_len = 512

base_model_path = "../models/phi_pubmed_pretrained_attempt_3/final"

model_id = "microsoft/Phi-3.5-mini-instruct"

model_output_dir = "../models/phi_qna_finetuned_attempt_1"

# Hyperparameters

In [47]:
lora_r = 16
lora_alpha = 32
lora_target_modules = ["q_proj", "v_proj", "o_proj"]
batch_size = 32
quantization = None
lora_dropout = 0.05
epochs = 5
learning_rate = 5e-5

# Dataset

In [4]:
prompt_template = """
# Instruction:
Assume you are an excellent doctor. Using your knowledge, answer the quesion given below.

# Question: {question}

# Answer: """
prompt_template = prompt_template.strip()
print(prompt_template)

# Instruction:
Assume you are an excellent doctor. Using your knowledge, answer the quesion given below.

# Question: {question}

# Answer:


In [45]:
def tokenize(example):
    prompt = prompt_template.format(question=example['question'])
    answer = example["answer"] + tokenizer.eos_token
    full_text = prompt + answer
    
    # Tokenize prompt to get its length
    prompt_tokens = tokenizer(
        prompt,
        truncation=False
    )
    
    prompt_len = len(prompt_tokens["input_ids"])

    # Tokenize full sequence once to get the total token count
    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=max_len
    )
    
    # Convert to numpy arrays for faster operations
    full_text_len = len(tokenized["input_ids"])
    
    # Tokenize full sequence once
    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=max_len,
        padding="max_length",
        return_attention_mask=True    
    )
    
    # Convert to numpy arrays for faster operations
    input_ids = np.array(tokenized["input_ids"])
    attention_mask = np.array(tokenized["attention_mask"])
    
    # Create labels array and mask prompt portion efficiently
    labels = input_ids.copy()
    # Mask the prompt tokens
    padding_len = max_len - full_text_len
    labels[padding_len:padding_len + prompt_len] = -100
    # print(full_text_len, padding_len, prompt_len)
    
    # Update the tokenized dict with numpy arrays
    tokenized["input_ids"] = input_ids.tolist()
    tokenized["attention_mask"] = attention_mask.tolist()
    tokenized["labels"] = labels.tolist()
    
    return tokenized

def tokenize_dataset(tokenizer, data_df):
    dataset = Dataset.from_pandas(data_df)
    dataset = dataset.map(tokenize, batched=False)
    return dataset

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

In [29]:
tokenizer([tokenizer.pad_token])

{'input_ids': [[32000]], 'attention_mask': [[1]]}

In [46]:
val_df = pd.read_csv(val_data_path)
train_df = pd.read_csv(train_data_path)

val_set = tokenize_dataset(tokenizer, val_df)
train_set = tokenize_dataset(tokenizer, train_df)

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 110000/110000 [02:56<00:00, 624.01 examples/s]


In [44]:
val_set[0]

{'question': 'Hi, may I answer your health queries right now ?  Please type  My son has developed a dime-sized soft moveable knot on back of his neck by the hairline? He has been doing some strenuous running in cross country-could this be the cause or should I have it checked out?',
 'answer': 'Hi, dairy have gone through your question. I can understand your concern. He may have some enlarged lymph node due to reactive hyperplasia or some soft tissue tumor like lipoma. He should go for examination and if needed go for biopsy of that lump. It will give you exact diagnosis. Then you should take treatment accordingly. Hope I have answered your question, if you have doubt then I will be happy to answer. Thanks for using Chat Doctor. Wish you a very good health.',
 'input_ids': [32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
  32000,
 

In [48]:
wandb.init(
    project="med-qna-finetune",
    name="attempt_1",
    config={
        "model": model_id,
        "lora_r": lora_r,
        "lora_alpha": lora_alpha,
        "batch_size": batch_size,
        "epochs": epochs,
        "quantization": quantization,
        "lora_target_modules": lora_target_modules
    }
)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hasindumadushan325 (hasindumadushan325-university-of-peradeniya) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=quantization=="4bit",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [53]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.89it/s]


In [54]:
model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

In [55]:
lora_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

/home/super_admin/hasindu/myenv/lib/python3.10/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/home/super_admin/hasindu/myenv/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


# Train

In [56]:
training_args = TrainingArguments(
    output_dir=model_output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    eval_strategy="epoch",  # ✅ eval at each epoch
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=learning_rate,
    fp16=True,
    report_to="wandb",
    run_name="attempt_1",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

In [57]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

/tmp/ipykernel_35596/2640495557.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

trainer.save_model(model_output_dir + "/final")
tokenizer.save_pretrained(model_output_dir + "/tokenizer")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
You are not running the flash-attention implementation, expect numerical differences.


Epoch,Training Loss,Validation Loss
